## **Persiapan Data**

In [ ]:
# Impor Library
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [ ]:
# Input data
data = pd.read_excel('/content/Preprocessed Clothing Dataset for Second-Hand Fashion.xlsx')

In [ ]:
# Mengisi nilai yang hilang
imputer = SimpleImputer(strategy='most_frequent')
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

In [ ]:
# Mengubah kolom kategorikal menjadi string
categorical_features = ['category', 'type', 'colors', 'season', 'cut', 'pattern', 'trend', 'smell', 'stains', 'holes', 'damage', 'material', 'comment', 'usage', 'Brand']
for col in categorical_features:
    data_imputed[col] = data_imputed[col].astype(str)

In [ ]:
# Encoding fitur kategorikal menjadi numerik
label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    data_imputed[col] = le.fit_transform(data_imputed[col])
    label_encoders[col] = le

In [ ]:
# Normalisasi fitur numerik
numerical_features = ['size', 'pilling', 'condition', 'price', 'annotator']
scaler = StandardScaler()
data_imputed[numerical_features] = scaler.fit_transform(data_imputed[numerical_features])

In [ ]:
data_imputed

,category,type,size,colors,season,pilling,condition,price,annotator,cut,pattern,trend,smell,stains,holes,damage,material,comment,usage,Brand
0,1,2,-0.216304,375,0,0.792598,-1.259974,-0.762189,-0.407448,8,2,4,0,1,1,29,27,16,1,1
1,1,1,-0.216304,13,0,0.792598,-1.259974,-0.213802,-0.407448,5,13,4,0,1,2,29,38,16,1,362
2,1,4,0.134322,13,0,-3.483159,-1.259974,-1.021951,-0.407448,14,7,4,1,2,2,29,169,16,2,201
3,0,28,5.569029,330,4,-1.345280,-0.407893,0.132548,0.757452,58,11,4,1,0,2,29,148,16,0,105
4,2,18,-0.286429,400,3,-0.276341,-0.407893,0.074823,0.757452,58,12,4,1,1,2,29,55,16,0,381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3047,1,4,-0.040991,13,0,-0.276341,0.444188,-0.300389,0.757452,31,11,4,2,0,2,29,754,16,0,240
3048,1,1,-0.251366,336,0,0.792598,0.444188,0.565485,0.757452,8,11,4,2,2,2,29,63,16,3,240
3049,1,17,-0.216304,336,0,-0.276341,-0.407893,0.045961,0.757452,59,11,4,2,0,2,29,530,16,0,182
3050,1,2,-0.216304,336,0,-1.345280,-0.407893,-0.184939,0.757452,58,11,4,2,2,2,29,55,16,0,240


## **Membuat Model Rekomendasi**

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Membuat fungsi untuk menghitung kesamaan
def calculate_similarity(data):
    similarity_matrix = cosine_similarity(data)
    return similarity_matrix

In [ ]:
# Menghitung kesamaan
similarity_matrix = calculate_similarity(data_imputed)
similarity_matrix

array([[1.        , 0.05392113, 0.10863233, ..., 0.57281766, 0.81161297,
        0.48467329],
       [0.05392113, 1.        , 0.82953743, ..., 0.38229888, 0.61424347,
        0.88375597],
       [0.10863233, 0.82953743, 1.        , ..., 0.75975821, 0.57032197,
        0.72242794],
       ...,
       [0.57281766, 0.38229888, 0.75975821, ..., 1.        , 0.68649741,
        0.53042205],
       [0.81161297, 0.61424347, 0.57032197, ..., 0.68649741, 1.        ,
        0.89656329],
       [0.48467329, 0.88375597, 0.72242794, ..., 0.53042205, 0.89656329,
        1.        ]])

In [ ]:
class RecommenderModel(tf.keras.Model):
    def __init__(self, similarity_matrix, **kwargs):
        super(RecommenderModel, self).__init__(**kwargs)
        self.similarity_matrix = tf.constant(similarity_matrix, dtype=tf.float32)

    def call(self, inputs):
        idx = tf.argmax(inputs, axis=1)
        return tf.gather(self.similarity_matrix, idx)

    def get_config(self):
        config = super(RecommenderModel, self).get_config()
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [ ]:
# Tentukan bentuk input
input_shape = (None, len(data_imputed))

# Pastikan untuk memanggil metode build sebelum konversi
model = RecommenderModel(similarity_matrix)
model.build(input_shape)

model

In [ ]:
# Bungkus metode panggilan dengan tf.function untuk memastikan TensorFlow memahami bentuk input
@tf.function(input_signature=[tf.TensorSpec(shape=[None, len(data_imputed)], dtype=tf.float32)])
def model_fn(inputs):
    return model(inputs)

In [ ]:
# Konversi model ke TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_concrete_functions([model_fn.get_concrete_function()])
tflite_model = converter.convert()

In [ ]:
# Menyimpan model
with open('recommender_model.tflite', 'wb') as f:
    f.write(tflite_model)

## **Tes Model Rekomendasi**

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
# Fungsi untuk memuat model TensorFlow Lite
def load_tflite_model(model_path):
    # Load TFLite model and allocate tensors.
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    return interpreter, input_details, output_details

# Fungsi untuk mendapatkan rekomendasi
def get_recommendations(interpreter, input_details, output_details, product_index, top_k=5):
    # Prepare input data
    input_data = np.zeros((1, len(data_imputed)))
    input_data[0, product_index] = 1  # Set the product index to 1 to indicate this is the product of interest

    # Set the tensor to point to the input data to be inferred
    interpreter.set_tensor(input_details[0]['index'], input_data.astype(np.float32))

    # Run inference
    interpreter.invoke()

    # The function `get_tensor()` returns a copy of the tensor data.
    # Use `tensor()` in order to get a pointer to the tensor.
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # Get top K recommendations
    recommendations = np.argsort(output_data[0])[::-1][:top_k]

    return recommendations

In [ ]:
# Path ke model TensorFlow Lite yang telah disimpan
model_path = '/content/recommender_model.tflite'

# Muat model TensorFlow Lite
interpreter, input_details, output_details = load_tflite_model(model_path)

In [ ]:
# Pilih indeks produk untuk rekomendasi (misalnya, produk pertama dalam data)
product_index = 0

# Dapatkan rekomendasi
recommendations = get_recommendations(interpreter, input_details, output_details, product_index, top_k=5)

# Tampilkan rekomendasi
print("Rekomendasi untuk produk:", product_index)
print("Indeks produk yang direkomendasikan:", recommendations)
print("Detail produk yang direkomendasikan:")
print(data.iloc[recommendations])

Rekomendasi untuk produk: 0
Indeks produk yang direkomendasikan: [   0 2334 1591 2192 2377]
Detail produk yang direkomendasikan:
     category      type  size        colors  season  pilling  condition  \
0      Ladies  Cardigan  38.0           Red     All        5          2   
2334   Ladies       Top  36.0         White     All        5          1   
1591   Ladies       Top  39.6  White Orange     All        5          2   
2192   Ladies    Blazer  40.0          Pink  Spring        5          5   
2377   Ladies       Top  38.0         White     All        5          1   

      price  annotator              cut     pattern     trend  smell stains  \
0        24       14.0  Ccollar Regular        Dots  No trend  Major  Minor   
2334     11        3.0           Collar       Plain  No trend     No  Major   
1591     16       14.8           Collar     Striped  No trend     No  Major   
2192    113       10.8           Collar       Plain  No trend     No     No   
2377     45        3.0   

In [ ]:
# Simpan hanya bobot model dalam format .h5
model.save_weights('recommender_model_weights.h5')